# Import Library

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('seaborn-darkgrid')
plt.rcParams['figure.figsize'] = (15, 13)
import seaborn as sns
sns.set_style('darkgrid')
from plotly import express as px, graph_objects as go

from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import gc
gc.enable()

# Mengambil Data

In [ ]:
%%time
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')

### Menghapus kolom yang tidak diperlukan

In [ ]:
train.drop(['Id', 'Soil_Type7', 'Soil_Type15'], axis = 1, inplace = True)
test.drop(['Id', 'Soil_Type7', 'Soil_Type15'], axis = 1, inplace = True)

### Ada label yang hanya mengandung 1 sampel, maka di-drop saja

In [ ]:
train = train.loc[train.Cover_Type != 5].reset_index(drop = True)
train

### Meng-encode label supaya bisa dibaca oleh komputer

In [ ]:
le = LabelEncoder()
train['Cover_Type'] = le.fit_transform(train['Cover_Type'])

### Function untuk mengurangi penggunaan memori karena data ini memakai > 3GB memori

In [ ]:
def reduce_memory_usage(df, verbose=True):
    from tqdm.auto import tqdm
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

# Modelling

In [ ]:
y = train.pop('Cover_Type')
y

### Memisah data yang untuk dibaca dan data yang untuk divalidasi

In [ ]:
FOLDS = 15
skf = StratifiedKFold(n_splits = FOLDS, random_state = 5, shuffle = True)

In [ ]:
help(RandomForestClassifier) # Dokumentasi Random Forest, berisi parameter apa saja yang bisa diubah-ubah supaya bisa mengoptimalkan akurasi.

In [ ]:
test_preds = []
val_preds = pd.Series()
scores = []
for i, (t, v) in enumerate(skf.split(train, y)) :
    xtrain = train.loc[t, :]
    xval = train.loc[v, :]
    xtest = test.copy()
    ytrain = y.loc[t]
    yval = y.loc[v]
    
    rf = RandomForestClassifier(random_state = 5, n_jobs = -1, verbose = 2, max_depth = None, n_estimators = 150)
    rf.fit(xtrain, ytrain)
    yhat = pd.Series(rf.predict(xval), index = v)
    val_preds = val_preds.append(yhat)
    ypred = rf.predict(xtest)
    test_preds.append(ypred)
    score = accuracy_score(yval, yhat)
    scores.append(score)
    print(f'Accuracy {i} : {score}')
    del xtrain
    del xval
    del xtest
    del ytrain
    del yval
    del rf
    gc.collect()
val_preds = val_preds.sort_index()
from scipy.stats import mode
train['rf_preds'] = val_preds
test['rf_preds'] = mode(test_preds, axis = 0)[0][0]
train.to_csv('rf_train.csv', index = False)
test.to_csv('rf_test.csv', index = False)

### Merata - rata hasil dari prediksi menggunakan K-Fold

In [ ]:
from scipy.stats import mode
ypreds = mode(test_preds, axis = 0)[0][0]
sample['Cover_Type'] = le.inverse_transform(ypreds)
sample.to_csv('submission.csv', index = False)

Semangat ;)